# Seinfeld Scritp Scraper 

### URL : http://www.seinology.com/scripts.shtml

In [1]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep
import pandas as pd
from itertools import chain
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

In [2]:
def extract_script(url):
    possible_scripts = []
    r = urllib.request.urlopen(url)
    parser = html.fromstring(r.read())
    for i in parser.xpath('//*[@id="content"]/p'):
        if (not("Looking for  a great gift idea for the h" in str(i.text_content())) and str(i.text_content()) != "" and len(str(i.text_content())) > 1):
            possible_scripts.append(str(i.text_content()))
    try:
        possible_scripts.append(str(parser.xpath("//pre//text()")))
    except:
        x = 1
    return possible_scripts

def extract_max(a_list):
    try:
        return " ".join(a_list)
    except:
        max_len = ""
        for i in a_list:
            if len(max_len) < len(i):
                max_len = i
        return max_len

In [3]:
response = get("http://www.seinfeldscripts.com/seinfeld-scripts.html")
cleaned_response = response.text.replace('\x00', "")
                                         
parser = html.fromstring(cleaned_response)
                                         
for i in parser.xpath('//table'):
    if i.attrib['width'] == "670":
        table_of_seasons = i  
                                         
tables = table_of_seasons.xpath('//table')

seasons = []
holder = ""
for i in tables[2:]:
    season = {}
    episodes = []
    if len(i.xpath(".//big//font//b")) > 0:
        season["season"] = i.xpath(".//big//font//b")[0].text
        holder = i.xpath(".//big//font//b")[0].text
    else:
        if (i.xpath(".//tr")[0].xpath(".//td")[0].text_content() !=
           "\xa0"):
            season["season"] = holder
            for j in i.xpath(".//tr"):
                episode = {}
                for k in j.xpath(".//td"):
                    episode["name"] = k.text_content()
                for k in j.xpath(".//td//a"):
                    episode["url"] = k.attrib["href"]
                episodes.append(episode)
    for j in i.xpath(".//tr")[1:]:
        episode = {}
        for k in j.xpath(".//td"):
            episode["name"] = k.text_content()
        for k in j.xpath(".//td//a"):
            episode["url"] = k.attrib["href"]
        episodes.append(episode)
    season["episodes"] = episodes
    seasons.append(season)

In [ ]:
df = pd.DataFrame(seasons)
df = (df['episodes'].apply(lambda x: pd.Series(x))
            .stack()
            .reset_index(level=1, drop=True)
            .to_frame('episodes')
            .join(df[['season']], how='left'))
df[["name","url"]] = df.episodes.apply(pd.Series)
df = df.drop("episodes",axis=1).reset_index(drop=True)
df = df.drop_duplicates().reset_index(drop=True).dropna()
df["url"] = df["url"].str.strip()
df["url"] = "http://www.seinfeldscripts.com/" + df["url"]
df["season_year"] = df["season"].str.extract("\(([^\)]+)\)")
df["season"] = df["season"].str.extract("^(.*?)\(")
df["name"] = df["name"].str.replace("\n","")
df["name"] = df["name"].str.strip()
df["date"] = df["name"].str.extract("\(([^\)]+)\)$")
df["name"] = df["name"].str.extract("(^(.*?)\(\d\)|.+?(?=\())")
df["script"] = df["url"].apply(extract_script)
df["big_script"] = df["script"].apply(extract_max)

In [ ]:
df.to_csv("seinfeld.csv",index=False)

In [ ]:
df